# ProbFuzz Artifact

Artifact of the paper "Testing Probabilistic Programming System" from ESEC/FSE ’18.

To access ChameleonCloud resources, you may need the account to log in to ChameleonCloud. You also need to have a project to allocate resources (e.g., node).

In [1]:
from chi import context

context.version = "1.0"

context.choose_site(default="CHI@TACC")
context.choose_project()

### Check available hardware

In [2]:
from chi import hardware

node_type = "compute_cascadelake_r"
available_nodes = hardware.get_nodes(node_type=node_type, filter_reserved=True)
if available_nodes:
    print(f"There currently are {len(available_nodes)} {node_type} nodes ready to use")
else:
    print(f"All {node_type} nodes are in use! You could use next_free_timeslot to see how long you need to wait, or use the calendar.")

There currently are 8 compute_cascadelake_r nodes ready to use


### Reserve node

In [3]:
from chi import lease
from datetime import timedelta
import os

my_lease = lease.Lease(f"{os.getenv('USER')}-power-management", duration=timedelta(hours=3))
my_lease.add_node_reservation(nodes=[available_nodes[0]]) # or you could use node_type=node_type
my_lease.add_fip_reservation(1) # include a floating ip
my_lease.submit(idempotent=True)

Waiting for lease to start... This can take up to 60 seconds


Lease radhofanazizi_gmail_com-power-management has reached status active


### Create a server on the node

In [4]:
from chi import server

my_server = server.Server(
    f"{os.getenv('USER')}-power-management",
    reservation_id=my_lease.node_reservations[0]["id"],
    image_name="CC-Ubuntu22.04", # or use image_name
)
my_server.submit(idempotent=True)

Waiting for server radhofanazizi_gmail_com-power-management's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,radhofanazizi_gmail_com-power-management
Id,ca0316f6-4282-44cd-b6ef-d0aa4ead63bb
Status,ACTIVE
Image Name,CC-Ubuntu22.04
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.2.100 (v4) Type: fixed MAC: bc:97:e1:c4:30:80
Network Name,sharednet1
Created At,2025-03-07T08:41:03Z
Keypair,trovi-3605188
Reservation Id,1519fc2a-2f17-49a9-8252-f33306ce386d
Host Id,b281b13a05d4a4d342f673906de4005142c2819a049809e34ac97306


### Configure networking on the node

In [5]:
fip = my_lease.get_reserved_floating_ips()[0]
my_server.associate_floating_ip(fip)
my_server.check_connectivity(host=fip)

Checking connectivity to 129.114.109.229 port 22.


Connection successful


## Run Probfuzz

Now, we can finally run Probfuzz. First we need to clone the github repo first and then run the reproduce.sh script which contain the instructions from README.md
packaged into a bash file

In [41]:
my_server.execute("git clone https://github.com/radhofan/probfuzz.git")

Cloning into 'probfuzz'...


<Result cmd='git clone https://github.com/radhofan/probfuzz.git' exited=0>

In [42]:
my_server.execute("chmod +x probfuzz/reproduce.sh")
my_server.execute("bash probfuzz/reproduce.sh")

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:4 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 398 kB in 1s (283 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
109 packages can be upgraded. Run 'apt list --upgradable' to see them.


Reading package lists...
Building dependency tree...
Reading state information...
openjdk-17-jdk is already the newest version (17.0.14+7-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 109 not upgraded.


openjdk version "17.0.14" 2025-01-21
OpenJDK Runtime Environment (build 17.0.14+7-Ubuntu-122.04.1)
OpenJDK 64-Bit Server VM (build 17.0.14+7-Ubuntu-122.04.1, mixed mode, sharing)




Reading package lists...
Building dependency tree...
Reading state information...
The following packages were automatically installed and are no longer required:
  libpython2-stdlib python-pkg-resources python-setuptools python2
  python2-minimal
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
  python3-wheel
The following packages will be REMOVED:
  python-pip
The following NEW packages will be installed:
  python3-pip python3-wheel
0 upgraded, 2 newly installed, 1 to remove and 109 not upgraded.
Need to get 1338 kB of archives.
After this operation, 2289 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-wheel all 0.37.1-2ubuntu0.22.04.1 [32.0 kB]
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-pip all 22.0.2+dfsg-1ubuntu0.5 [1306 kB]


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


...

--2025-03-07 09:22:25--  http://www.antlr.org/download/antlr-4.7.1-complete.jar
Resolving www.antlr.org (www.antlr.org)... 185.199.110.153, 185.199.109.153, 185.199.111.153, ...
Connecting to www.antlr.org (www.antlr.org)|185.199.110.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.antlr.org/download/antlr-4.7.1-complete.jar [following]
--2025-03-07 09:22:25--  https://www.antlr.org/download/antlr-4.7.1-complete.jar
Connecting to www.antlr.org (www.antlr.org)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2240116 (2.1M) [application/java-archive]
Saving to: ‘probfuzz/language/antlr/antlr-4.7.1-complete.jar’

     0K .......... .......... .......... .......... ..........  2% 2.92M 1s
    50K .......... .......... .......... .......... ..........  4% 3.94M 1s
   100K .......... .......... .......... .......... ..........  6% 14.9M 0s
   150K .......... .......... .......... .......... ....

Install successful


/home/cc/probfuzz/backends/edward.py:155: SyntaxWarning: import * only allowed at module level
  def visitTemplate(self, ctx):


Output directory : probfuzz/output/progs20250307-092228
Starting thread .. 1
Running Stan program 1 >>>>
Done Stan program 1 >>>>
Running Edward program 1 >>>>
Done Edward program 1 >>>>
Running Pyro program 1 >>>>
Done Pyro program 1 >>>>
Running Stan program 2 >>>>
Done Stan program 2 >>>>
Running Edward program 2 >>>>
Done Edward program 2 >>>>
Running Pyro program 2 >>>>
Done Pyro program 2 >>>>
Running Stan program 3 >>>>
Done Stan program 3 >>>>
Running Edward program 3 >>>>
Done Edward program 3 >>>>
Running Pyro program 3 >>>>
Done Pyro program 3 >>>>
Running Stan program 4 >>>>
Done Stan program 4 >>>>
Running Edward program 4 >>>>
Done Edward program 4 >>>>
Running Pyro program 4 >>>>
Done Pyro program 4 >>>>
Running Stan program 5 >>>>
Done Stan program 5 >>>>
Running Edward program 5 >>>>
Done Edward program 5 >>>>
Running Pyro program 5 >>>>
Done Pyro program 5 >>>>
Printing summary
Summary written in summary.csv


<Result cmd='bash probfuzz/reproduce.sh' exited=0>

Check the correct program folder to view the results

In [43]:
my_server.execute("ls -l probfuzz/output")

total 4
drwxrwxr-x 7 cc cc 4096 Mar  7 09:22 progs20250307-092228


<Result cmd='ls -l probfuzz/output' exited=0>

Place the correct program folder in probfuzz/output/[program folder]/summary.csv

In [44]:
my_server.execute("cat probfuzz/output/progs20250307-092228/summary.csv")

Program       Stan_Crash  Stan_Num  Stan_Acc  Pyro_Crash  Pyro_Num  Pyro_Acc  Edward_Crash  Edward_Num  Edward_Acc
prob_rand_1/  *           -         -         *           -         -         *             -           -
prob_rand_2/  *           -         -         *           -         -         *             -           -
prob_rand_3/  *           -         -         *           -         -         *             -           -
prob_rand_4/  *           -         -         *           -         -         *             -           -
prob_rand_5/  *           -         -         *           -         -         *             -           -


<Result cmd='cat probfuzz/output/progs20250307-092228/summary.csv' exited=0>